In [25]:

# imports
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense, BatchNormalization
import matplotlib.pyplot as plt 
import sklearn.model_selection
from sklearn.model_selection import train_test_split
from random import random, shuffle
from sklearn.metrics import accuracy_score
from python_speech_features import mfcc
from braindecode.models import Deep4Net
import antropy
import sys
from scipy import integrate, stats
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsRegressor
from python_speech_features import mfcc
from scipy import spatial

from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor




In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
from sklearn.neighbors import KNeighborsClassifier

In [15]:
def tran(x, label1):
    if x == label1:
        return 1
    else:
        return 0

def get_data(type_of_data = 'imagined', subject_num = 1):
    '''Gets the data from the directory given the type of data
    and which subject to get it from. Returns an array (raw_eeg, labels)'''
    raw_eeg, all_labels = np.genfromtxt(f'{type_of_data}/labelled/{type_of_data}{subject_num}-labelled.csv', delimiter=',', dtype='float')[1:, 0:14], np.genfromtxt(f'{type_of_data}/labelled/{type_of_data}{subject_num}-labelled.csv', delimiter=',', dtype='str')[1::256, -1]
    raw_eeg = raw_eeg.reshape(1000, 256, 14)
    return raw_eeg, all_labels

In [8]:
def mean(x):
	return np.mean(x)

def absmean(x):
	return np.mean(np.abs(x))

def maximum(x):
	return np.max(x)

def absmax(x):
	return np.max(np.abs(x))

def minimum(x):
	return np.min(x)

def absmin(x):
	return np.min(np.abs(x)) 

def minplusmax(x):
	return np.max(x) + np.min(x)

def maxminusmin(x):
	return np.max(x) - np.min(x)

def curvelength(x):
	cl = 0
	for i in range(x.shape[0]-1):
		cl += abs(x[i]-x[i+1])
	return cl

def energy(x):
	return np.sum(np.multiply(x,x))

def nonlinear_energy(x):
	# NLE(x[n]) = x**2[n] - x[n+1]*x[n-1]
	x_squared = x[1:-1]**2
	subtrahend = x[2:]*x[:-2]
	return np.sum(x_squared-subtrahend)
	

def spec_entropy(x):
	return antropy.spectral_entropy(x, 128,method="welch",normalize=True)

def integral(x):
	return integrate.simps(x)

def stddeviation(x):
	return np.std(x)

def variance(x):
	return np.var(x)

def skew(x):
	return stats.skew(x)

def kurtosis(x):
	return stats.kurtosis(x)

def get_mfcc(x):
	return mfcc(x)


#doesn't contain EHF since that must be added later
funclist = [mean,absmean,maximum,absmax,minimum,absmin,minplusmax,maxminusmin,curvelength,energy,nonlinear_energy, integral,stddeviation,variance,skew,np.sum,spec_entropy, kurtosis] # , sample_entropy, perm_entropy, svd_entropy, app_entropy,petrosian, katz, higuchi, rootmeansquare] 


In [9]:
def binary_split(data, labels, label1, label2):
    new_features, new_labels = data[(np.array(labels) == label1) + (np.array(labels) == label2)].astype('float32'), np.array(labels)[(np.array(labels) == label1) + (np.array(labels) == label2)]
    new_labels1 = [tran(x, label1) for x in new_labels]
    return new_features, new_labels1

In [10]:
def get_windows(raw_eeg, all_labels, raw = False):
    '''Gets the windows for our eeg data, 15 windows per epoch, length 32 windows. EEG data must be split binary.
    Returns the windowed data, the labels for each window and the feature array, and the labels for each feature array'''
    if raw == True:
        raw_eeg = raw_eeg.reshape(len(all_labels), 256, 14)
        labelled_data = []
        feature_array = []
        all_windows = []
        window_labels = []
        for i in range(raw_eeg.shape[0]):
            epoch = raw_eeg[i]
            label = all_labels[i]
            for j in range(15):
                window = np.array(epoch[(j * 16):((j + 2)*16)])
                all_windows.append(window)
                window_labels.append(label)

        return np.array(all_windows), window_labels

    else:
        raw_eeg = raw_eeg.reshape(len(all_labels), 256, 14)
        labelled_data = []
        feature_array = []
        for i in range(raw_eeg.shape[0]):
            epoch = raw_eeg[i]
            label = all_labels[i]
            windowed_data = []
            for j in range(15):
                window = np.array(epoch[(j * 16):((j + 2)*16)])
                outvec = np.zeros((len(funclist), window.shape[1]))
                for i, f in enumerate(funclist):
                    # print('applying {}'.format(f.__name__))
                    for col in range(window.shape[1]):
                        outvec[i, col] = f(window[:, col])
                outvec = outvec.reshape(-1)
                windowed_data.append(outvec)

            # 1-> 5 ... 11 -> 15
            for k in range(11):
                feature_array.append(np.stack(windowed_data[(k):(k+5)]))
                labelled_data.append(label)

        feature_array = np.array(feature_array)
        return feature_array, np.array(labelled_data)

In [17]:
label_pairs = [('goose', 'moose'), ('moose', 'spruce'),
                ('moose', 'caterpillar'), ('juice', 'waterfowl'),
                ('hedgehog', 'goose'), ('night', 'knight'), ('date', 'juice'), 
                ('hedgehog', 'waterfowl'), ('juice', 'ambassador')]

kfold = sklearn.model_selection.StratifiedKFold(n_splits = 5, shuffle = True, random_state=1)


In [17]:
# knn
parameters = {"n_neighbors": range(50, 200)}
kfold = sklearn.model_selection.StratifiedKFold(n_splits = 5, shuffle = True, random_state=1)
pca = PCA(0.95)
# function to import data and divide
def knn_dist_test(data, labels, label1, label2):
    data = data.reshape(1000, 256, 14)
   
    feats, labs = binary_split(data, labels, label1, label2)
    labs = np.array(labs)

    feats = feats.reshape(feats.shape[0], feats.shape[1] * feats.shape[2])
    val_accuracies = []
    for train_ind, val_ind in list(kfold.split(feats, labs)):
        feats_train, ft_labels = get_windows(feats[train_ind], labs[train_ind])
        feats_val, fv_labels = get_windows(feats[val_ind], labs[val_ind])

        feats_train = feats_train.reshape(feats_train.shape[0], feats_train.shape[1] * feats_train.shape[2])
        feats_val = feats_val.reshape(feats_val.shape[0], feats_val.shape[1] * feats_val.shape[2])
        #standardize now...
        feats_train = np.nan_to_num(feats_train)
        feats_val = np.nan_to_num(feats_val)
        feats_train, feats_val = StandardScaler().fit_transform(feats_train), StandardScaler().fit_transform(feats_val)
        # perform pca on training features
        pca.fit(feats_train)
        feats_train, feats_val = pca.transform(feats_train), pca.transform(feats_val)

        
        gridsearch = GridSearchCV(KNeighborsClassifier(), parameters)
        gridsearch.fit(feats_train, ft_labels)
        # gridsearch = KNeighborsClassifier(n_neighbors=95)
        # gridsearch.fit(feats_train, ft_labels)
        #predict our test set and record a score
        preds = [np.round(i) for i in gridsearch.predict(feats_val)]
        val_accuracies.append(accuracy_score(fv_labels, preds))

    print('KNN Binary classification between {} and {} for subject {}, {}'.format(label1, label2, subject_num, data_type))
    print('================================================================================================')
    print('10 fold Cross-validated accuracy: {} +- {}'.format(np.mean(val_accuracies), np.std(val_accuracies)))
    print()
    return 


for subject_num in range(1, 9):
    for data_type in ['imagined', 'inner']:
        print('Subject {}, {}'.format(subject_num, data_type))
        data, labels = get_data(data_type, subject_num)
        for label1, label2 in label_pairs:
            knn_dist_test(data, labels, label1, label2)


Subject 1, imagined
KNN Binary classification between goose and moose for subject 1, imagined
10 fold Cross-validated accuracy: 0.5663636363636363 +- 0.04967664033568819

KNN Binary classification between moose and spruce for subject 1, imagined
10 fold Cross-validated accuracy: 0.511818181818182 +- 0.05163444298604701



KeyboardInterrupt: 

In [30]:
def L4(x, y):
    return (np.sum((x - y)**4))**(1 / 4)

def L2(x, y):
    return (np.sum((x - y)**2))**(1 / 2)

def L3(x, y):
    return (np.sum((x - y)**3))**(1 / 3)

def chi2_distance(x, y):
 
    # compute the chi-squared distance using above formula
    chi = 0.5 * np.sum(((x - y) ** 2) / (x + y))
 
    return chi

def corr_coef(x, y):
    return 1 - np.corrcoef(x, y)[0, 1]

def spatial_dist(x, y):
    return spatial.distance.cosine(x, y)

funclist2 = [L4, L2, L3, chi2_distance, corr_coef, spatial_dist]

In [34]:
knn_metric_accs = np.zeros((4, 2, 9, 6))
knn_metric_stds = np.zeros((4, 2, 9, 6))
pca = PCA(0.95)
parameters = {"n_neighbors": range(90, 100)}

for subject_num in [5,6,7,8]:
    for d, data_type in enumerate(['imagined', 'inner']):
        data, labels = get_data(data_type, subject_num)
        for l ,(label1, label2) in enumerate(label_pairs):
            X, y = binary_split(data, labels, label1, label2)
            y = np.array(y)
            print('================================================================================================')
            print('5 fold Cross-validated accuracies KNN Binary classification between {} and {} for subject {}, {}'.format(label1, label2, subject_num, data_type))
            print('-------------------------------------------------------------------------')
            for f, func in enumerate(funclist2):
                val_accuracies = []
                for train_ind, val_ind in list(kfold.split(X, y)):
                    train_feats, train_labels = get_windows(X[train_ind], y[train_ind])
                    val_feats, val_labels = get_windows(X[val_ind], y[val_ind])

                    feats_train = train_feats.reshape(train_feats.shape[0], train_feats.shape[1] * train_feats.shape[2])
                    feats_val = val_feats.reshape(val_feats.shape[0], val_feats.shape[1] * val_feats.shape[2])
                    #standardize now...
                    feats_train = np.nan_to_num(feats_train)
                    feats_val = np.nan_to_num(feats_val)
                    feats_train, feats_val = StandardScaler().fit_transform(feats_train), StandardScaler().fit_transform(feats_val)
                    # perform pca on training features
                    pca.fit(feats_train)
                    feats_train, feats_val = pca.transform(feats_train), pca.transform(feats_val)

                    gridsearch = KNeighborsClassifier(metric=func)
                    gridsearch.fit(feats_train, train_labels)
                    # gridsearch = GridSearchCV(KNeighborsClassifier(metric = func), parameters)
                    # gridsearch.fit(feats_train, train_labels)
                    #predict our test set and record a score
                    preds = [np.round(i) for i in gridsearch.predict(feats_val)]
                    val_accuracies.append(accuracy_score(val_labels, preds))
                    # print('.')

                knn_metric_accs[subject_num - 5][d][l][f] +=  np.mean(val_accuracies)
                knn_metric_stds[subject_num - 5][d][l][f] +=  np.std(val_accuracies)
                print('Distance metric: {}, Accuracy +- sd: {} +- {}'.format(func.__name__, np.mean(val_accuracies), np.std(val_accuracies)))
                print()

5 fold Cross-validated accuracies KNN Binary classification between goose and moose for subject 5, imagined
-------------------------------------------------------------------------
Distance metric: L4, Accuracy +- sd: 0.5027272727272727 +- 0.07287485026847666

Distance metric: L2, Accuracy +- sd: 0.5154545454545454 +- 0.0817474441686049

Distance metric: L3, Accuracy +- sd: 0.5136363636363637 +- 0.04954336943068622

Distance metric: chi2_distance, Accuracy +- sd: 0.5036363636363637 +- 0.027783103254429294

Distance metric: corr_coef, Accuracy +- sd: 0.4763636363636364 +- 0.08731059784394885

Distance metric: spatial_dist, Accuracy +- sd: 0.48 +- 0.08280216390353236

5 fold Cross-validated accuracies KNN Binary classification between moose and spruce for subject 5, imagined
-------------------------------------------------------------------------
Distance metric: L4, Accuracy +- sd: 0.5990909090909091 +- 0.027783103254429318

Distance metric: L2, Accuracy +- sd: 0.5818181818181818 +- 0

In [38]:
knn_metric_accs[:, :, :, 0].shape

(4, 2, 9)

In [39]:
knn_metric_accs.shape

(4, 2, 9, 6)

In [44]:
knn_metric_accs[0][:, 0,3]

array([0.50363636, 0.49636364])

In [60]:
knn_accs = np.mean(np.mean(knn_metric_accs, axis = 0), axis = 0)

In [61]:
knn_sds = np.mean(np.mean(knn_metric_stds, axis = 0), axis = 0)

In [63]:
for i, row in enumerate(knn_accs):
    print('& {:.2f} $\\pm$ {:.2f} & {:.2f} $\\pm$ {:.2f} & {:.2f} $\\pm$ {:.2f} & {:.2f} $\\pm$ {:.2f} & {:.2f} $\\pm$ {:.2f} & {:.2f} $\\pm$ {:.2f} \\\\'.format(row[0], knn_sds[i][0],row[1], knn_sds[i][1],row[2], knn_sds[i][2], row[3], knn_sds[i][3], row[4], knn_sds[i][4], row[5], knn_sds[i][5]))

& 0.55 $\pm$ 0.05 & 0.54 $\pm$ 0.06 & 0.52 $\pm$ 0.03 & 0.49 $\pm$ 0.03 & 0.54 $\pm$ 0.07 & 0.54 $\pm$ 0.07 \\
& 0.59 $\pm$ 0.05 & 0.59 $\pm$ 0.05 & 0.55 $\pm$ 0.04 & 0.50 $\pm$ 0.03 & 0.58 $\pm$ 0.05 & 0.58 $\pm$ 0.05 \\
& 0.56 $\pm$ 0.05 & 0.55 $\pm$ 0.05 & 0.54 $\pm$ 0.03 & 0.49 $\pm$ 0.03 & 0.56 $\pm$ 0.04 & 0.55 $\pm$ 0.04 \\
& 0.54 $\pm$ 0.05 & 0.54 $\pm$ 0.05 & 0.51 $\pm$ 0.03 & 0.50 $\pm$ 0.03 & 0.55 $\pm$ 0.05 & 0.54 $\pm$ 0.05 \\
& 0.55 $\pm$ 0.05 & 0.56 $\pm$ 0.05 & 0.52 $\pm$ 0.03 & 0.49 $\pm$ 0.03 & 0.56 $\pm$ 0.06 & 0.56 $\pm$ 0.06 \\
& 0.52 $\pm$ 0.06 & 0.53 $\pm$ 0.06 & 0.51 $\pm$ 0.04 & 0.49 $\pm$ 0.03 & 0.54 $\pm$ 0.05 & 0.54 $\pm$ 0.04 \\
& 0.54 $\pm$ 0.04 & 0.54 $\pm$ 0.05 & 0.50 $\pm$ 0.04 & 0.50 $\pm$ 0.03 & 0.54 $\pm$ 0.04 & 0.54 $\pm$ 0.04 \\
& 0.55 $\pm$ 0.06 & 0.55 $\pm$ 0.06 & 0.51 $\pm$ 0.04 & 0.50 $\pm$ 0.03 & 0.54 $\pm$ 0.06 & 0.54 $\pm$ 0.06 \\
& 0.54 $\pm$ 0.07 & 0.54 $\pm$ 0.06 & 0.52 $\pm$ 0.04 & 0.50 $\pm$ 0.02 & 0.54 $\pm$ 0.06 & 0.54 $\pm$ 0.06 \\


In [68]:
knn_metric_accs.shape

(4, 2, 9, 6)

In [71]:
np.mean(np.mean(knn_metric_accs, axis=2), axis = 1)

array([[0.53222222, 0.52833333, 0.52121212, 0.49651515, 0.53161616,
        0.53080808],
       [0.53641414, 0.5339899 , 0.51257576, 0.49808081, 0.52893939,
        0.52772727],
       [0.60343434, 0.60792929, 0.53222222, 0.49353535, 0.60449495,
        0.60383838],
       [0.52277778, 0.525     , 0.51373737, 0.49707071, 0.52994949,
        0.5290404 ]])

In [87]:
sd=  np.mean(np.mean(knn_metric_stds, axis=2), axis = 1).T
for i, row in enumerate(np.mean(np.mean(knn_metric_accs, axis=2), axis = 1).T):
    print('&{:.4f} $\\pm$ {:.4f} & {:.4f} $\\pm$ {:.4f} & {:.4f} $\\pm$ {:.4f} & {:.4f} $\\pm$ {:.4f}\\\\'.format(row[0],sd[i][0], row[1],sd[i][1], row[2],sd[i][2], row[3],sd[i][3]))

&0.5322 $\pm$ 0.0676 & 0.5364 $\pm$ 0.0432 & 0.6034 $\pm$ 0.0575 & 0.5228 $\pm$ 0.0461\\
&0.5283 $\pm$ 0.0672 & 0.5340 $\pm$ 0.0476 & 0.6079 $\pm$ 0.0544 & 0.5250 $\pm$ 0.0476\\
&0.5212 $\pm$ 0.0363 & 0.5126 $\pm$ 0.0327 & 0.5322 $\pm$ 0.0379 & 0.5137 $\pm$ 0.0374\\
&0.4965 $\pm$ 0.0303 & 0.4981 $\pm$ 0.0272 & 0.4935 $\pm$ 0.0285 & 0.4971 $\pm$ 0.0279\\
&0.5316 $\pm$ 0.0583 & 0.5289 $\pm$ 0.0480 & 0.6045 $\pm$ 0.0547 & 0.5299 $\pm$ 0.0499\\
&0.5308 $\pm$ 0.0572 & 0.5277 $\pm$ 0.0469 & 0.6038 $\pm$ 0.0550 & 0.5290 $\pm$ 0.0485\\


In [76]:
np.mean(np.mean(knn_metric_stds, axis=2), axis = 1)[0]

array([0.06761234, 0.06724207, 0.0363186 , 0.03031443, 0.0583314 ,
       0.05721514])

In [85]:
np.mean(np.mean(knn_metric_accs, axis=2), axis = 1).T

array([[0.53222222, 0.53641414, 0.60343434, 0.52277778],
       [0.52833333, 0.5339899 , 0.60792929, 0.525     ],
       [0.52121212, 0.51257576, 0.53222222, 0.51373737],
       [0.49651515, 0.49808081, 0.49353535, 0.49707071],
       [0.53161616, 0.52893939, 0.60449495, 0.52994949],
       [0.53080808, 0.52772727, 0.60383838, 0.5290404 ]])